In [1]:
from tqdm.auto import tqdm
import polars as pl

import sys
sys.path.insert(0, '/iblm/netapp/home/dlaub/projects/hyena-dna')

from src.dataloaders import gvl  # noqa: E402
from src.dataloaders import multifasta # noqa: E402

In [2]:
files = pl.read_csv('/iblm/netapp/home/dlaub/projects/hyena-dna/data/cactus_arrow_dataset_65536.txt', separator='\t')
some_files = files.slice(0, 10)
bed = pl.read_ipc('/iblm/netapp/home/dlaub/projects/hyena-dna/data/cactus_dataset_exactly_2to16_all_bed.arrow')

Pure GVL implementation is OOM/very slow to initialize.

In [3]:
dm = gvl.MultiFasta(
    some_files,
    max_length=65536,
    batch_size=16,
    max_memory_gb=16,
)

Initializing fastas:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
for batch in tqdm(dm.train_dataloader()):
    pass

0it [00:00, ?it/s]

ValueError: could not broadcast input array from shape (37716,) into shape (65536,)

In [ ]:
batch['seq']

In [10]:

species = some_files['fasta'].str.split('/').list.get(-1).str.split('.').list.get(0)
dm2 = multifasta.MultiFasta(
    file_table=some_files,
    bed=bed.filter(pl.col('species').is_in(species)),
    batch_size=128,
    num_workers=1,
    
)

Init FASTAs:   0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
dm2.setup()
for i, batch in tqdm(enumerate(dm2.train_dataloader()), total=len(dm2.train_dataloader())):
    if i == int(1e3):
        break

  0%|          | 0/748 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
for f in dm2.fastas.values():
    f.close()

In [8]:
batch['seq']

tensor([[10,  9,  8,  ..., 10, 10,  8],
        [10,  8, 10,  ...,  8,  9,  9],
        [ 9, 10,  7,  ...,  8, 10,  7],
        ...,
        [ 8,  8,  9,  ...,  7, 10,  7],
        [ 9,  9,  7,  ...,  9,  9,  9],
        [ 8,  7,  8,  ...,  8, 10,  9]], dtype=torch.int32)